In [1]:
!git clone 'https://github.com/aakashvardhan/s7-in-depth-practice.git'

Cloning into 's7-in-depth-practice'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 90 (delta 39), reused 75 (delta 24), pack-reused 0
Receiving objects: 100% (90/90), 195.39 KiB | 2.68 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [2]:
import sys
sys.path.append('/content/s7-in-depth-practice')
sys.path.append('/content/s7-in-depth-practice/models')

In [3]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from model_utils import model_summary, test_model_sanity, sgd_optimizer
from models.model import model3

from utils import train, test, get_device, plt_fig, get_incorrect_predictions, plot_incorrect

In [4]:
random_rotations = 7
img_size = (28,28)

# Train Phase transformations
train_transforms = transforms.Compose([
                                       transforms.RandomResizedCrop(img_size, (0.80, 1.00)),
                                       transforms.RandomRotation(random_rotations),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values.
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])

In [5]:
train_data = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test_data = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

100%|██████████| 9912422/9912422 [00:00<00:00, 63625223.23it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 40595071.66it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 31218053.11it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 6524153.69it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
# CUDA?
cuda = get_device()

# For reproducibility
torch.manual_seed(1)

if torch.cuda.is_available():
    torch.cuda.manual_seed(1)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train_data, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test_data, **dataloader_args)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
model3 = model3().to(cuda)

In [8]:
model_summary(model3)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 26, 26]              54
              ReLU-2            [-1, 6, 26, 26]               0
       BatchNorm2d-3            [-1, 6, 26, 26]              12
           Dropout-4            [-1, 6, 26, 26]               0
         ConvBlock-5            [-1, 6, 26, 26]               0
            Conv2d-6            [-1, 4, 24, 24]             216
              ReLU-7            [-1, 4, 24, 24]               0
       BatchNorm2d-8            [-1, 4, 24, 24]               8
           Dropout-9            [-1, 4, 24, 24]               0
        ConvBlock-10            [-1, 4, 24, 24]               0
           Conv2d-11           [-1, 12, 22, 22]             432
             ReLU-12           [-1, 12, 22, 22]               0
      BatchNorm2d-13           [-1, 12, 22, 22]              24
          Dropout-14           [-1, 12,

In [9]:
from torch.optim.lr_scheduler import StepLR


optimizer = sgd_optimizer(model3, 0.05)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)
EPOCHS = 15
lr = []
for epoch in range(1,EPOCHS+1):
    print("EPOCH:", epoch)
    train(model3, cuda, train_loader, optimizer, epoch)
    test(model3, cuda, test_loader)
    scheduler.step()
    lr.append(optimizer.param_groups[0]['lr'])

EPOCH: 1


Loss=0.2524404227733612 Batch_id=468 Accuracy=82.70: 100%|██████████| 469/469 [00:34<00:00, 13.71it/s]



Test set: Average loss: 0.1534, Accuracy: 9517/10000 (95.17%)

EPOCH: 2


Loss=0.21917130053043365 Batch_id=468 Accuracy=93.24: 100%|██████████| 469/469 [00:31<00:00, 14.76it/s]



Test set: Average loss: 0.0778, Accuracy: 9760/10000 (97.60%)

EPOCH: 3


...


Test set: Average loss: 0.0434, Accuracy: 9861/10000 (98.61%)

EPOCH: 14


Loss=0.1158754825592041 Batch_id=468 Accuracy=96.49: 100%|██████████| 469/469 [00:33<00:00, 14.17it/s]



Test set: Average loss: 0.0478, Accuracy: 9846/10000 (98.46%)

EPOCH: 15


Loss=0.03750893101096153 Batch_id=468 Accuracy=96.72: 100%|██████████| 469/469 [00:31<00:00, 14.80it/s]



Test set: Average loss: 0.0450, Accuracy: 9853/10000 (98.53%)



In [11]:
incorrect, incorrect_pred, incorrect_target = get_incorrect_predictions(model3, cuda, test_loader)

IndexError: too many indices for tensor of dimension 1

In [ ]:
# Save model
torch.save(model3.state_dict(), 'best_model.pth')